# DHd-Hackathon 2021 Challenge 3
Von Kai Krüger und Dennis Friedl (Universität Paderborn)

Der folgende Code trainiert ein Modell für Named-entity recognition in einem frühneuhochdeutschen Text. Dafür wird die Python-Bibliothek "Spacy" genutzt. Seit der Version 3 hat sich Spacy grundlegend geändert. Falls mit einer Spacy-Version unter 2 gearbeitet wird, kann der auskommentierte Code der dritten Zelle genutzt werden, um die Trainings-CSV in das von Spacy 2 erwartete Format zu bringen. Für Spacy 3 ist dieser Schritt nicht mehr notwendig.

In [ ]:
import spacy
from spacy.tokens import Doc, DocBin
import pandas as pd
#import re # nur für Spacy 2 notwenig

In [ ]:
# NUR FÜR SPACY 2:
# train = pd.read_csv('./Daten/train.csv')
# point_locations = train.loc[train.isin(['.']).any(axis=1)].index.tolist()
# list_of_sentences = []
# davor = 0 
# for p in point_locations:
#     list_of_sentences.append(train.iloc[davor:p+1, :])
#     davor = p+1
# list_of_sentences
# TRAIN_DATA = []
# for df in list_of_sentences:
#     tokens = df['Token'].tolist()
#     tags = df['Tag'].tolist()
#     satz = ' '.join(tokens)
#     info = []
#     for i in range(len(tokens)):
#         if tags[i] != "O":
#             for match in re.finditer(tokens[i], satz):
#                 s = match.start()
#                 e = match.end()
#                 dic1 = {"token": tokens[i], "tag": tags[i], "start": s, "end": e}
#                 if dic1 not in info:
#                     info.append(dic1)
#     tail = []
#     for ent in info:
#         tail.append((ent["start"], ent["end"], ent["tag"]))
#     sentence_with_info = (satz, {"entities": tail})
#     if sentence_with_info[1]["entities"]:
#         TRAIN_DATA.append(sentence_with_info)

## Bringt die Trainings-CSV in das von Spacy 3 erwartete Format:

In [ ]:
nlp = spacy.blank("de") # Deutschsprachiges Modell
docbin = DocBin()
df = pd.read_csv("./Daten/train.csv")

In [ ]:
words = df['Token'].tolist() # Alle Tokens in eine Liste
spaces = [False if w in '.,;:!?' else True for w in words] # Wann sollen Leerzeichen eingefügt werden und wann nicht?
ents = df['Tag'].tolist() # Alle Tags in eine Liste


In [ ]:
doc = Doc(nlp.vocab, words=words, spaces=spaces, ents=ents)
docbin.add(doc)
docbin.to_disk("./train.spacy")
# Output ist eine Spacy-Trainingsdatei

## Erstellen einer Config-Datei:

Die vielen Stellschrauben eines Modelltrainings müssen mit Spacy 3 nicht mehr in Python-Code modifiziert werden, sondern laufen über eine Konfigurationsdatei. Um diese Datei zu initialisieren, können auf der Webseite von Spacy https://spacy.io/usage/training zunächst einfache Parameter wie die Sprache und die Komponenten (in unserem Fall NER) eingestellt werden, bevor die Datei dann heruntergeladen werden kann. Diese Datei muss dann noch weiter mit dem Konsolenbefehl ```python -m spacy init fill-config base_config.cfg config.cfg``` abschließend initialisiert werden.

Die Konfigurationsdatei kann nun in einem Editor ganz normal geöffnet werden. Hier lohnt es sich mit Sicherheit, etwas mit den Werten und Einstellungen herumzuspielen, um die besten Parameter für frühneuzeitliche Texte herauszufinden. 

Auf was auf jeden Fall geachtet werden muss, ist dass in den Einstellungen für das Trainingsset die Parameter für "max_length" und "limit" auf jeweils "0", also unendlich, gesetzt sind. Wir hatten deswegen eine Fehlermeldung, die wir lange Zeit nicht zuordnen konnten.

 


## Trainieren des Modells:

Für das Training an sich braucht man tatsächlich kein Python mehr, denn die Bedienung von Spacy funktioniert seit Spacy 3 ganz einfach über die Konsole. Dafür gibt man, nachdem die Scpay-Trainingsdatei und die Config erstellt wurden, folgendes in die Konsole: ```python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy```

In diesem Fall läuft das Training über die CPU. Falls ein Training über GPU erwünscht ist, kann außerdem der Befehl ```--gpu-id 0``` angefügt werden. In der Konsole kann abgelesen werden, wie sich Daten wie Recall, Precision etc. im Laufe des Trainings ändern. Im Ordner "Output" finden sich jetzt sowohl das letzte als auch das beste Modell.

## Anwenden des Modells an der test.csv:

Das Modell kann nun an der bereitgestellten test.csv getestet werden.

### Taggen des Textes:

In [ ]:
nlp1 = spacy.load("./output/model-best") # lädt das beste Modell
df = pd.read_csv("./Daten/test.csv") # lädt die test.csv

doc = ' '.join(df["Token"].tolist()) # fügt die Tokens zu einem zusammenhängenden Text zusammen
doc = nlp1(doc) # Anwendung des Modells

spacy.displacy.render(doc, style="ent", jupyter=True) # Anzeige des getaggten Textes

### Übersetzen in das gewünschte Format (mit I- und B-Präfix):

In [ ]:
tokens = []    
ent_list_dirty = []
for t in doc: # Speichere Tokens und Tags in Liste
    tokens.append(t.text)
    ent_list_dirty.append(t.ent_type_)

ent_list = []
for i in range(len(ent_list_dirty)): # Anpassung der Tags als "O" oder mit dem I- oder B-Präfix
    if ent_list_dirty[i] == "":
       ent_list.append("O")
    else:
        if i > 1 and ent_list_dirty[i-1] == ent_list_dirty[i]:
            ent_list.append("I-" + ent_list_dirty[i])
        else:
            ent_list.append("B-" + ent_list_dirty[i])

### Speichern des getaggten Textes als CSV:

In [ ]:
output_df = pd.DataFrame(list(zip(tokens, ent_list)),
columns = ['Tokens', 'Tag'])

output_df.to_csv("test_output.csv")